In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
import os

template = """
You are a SQL expert. Based on the table schema below, write just the SQL
query without the results that would answer the user's question.:
{schema}
Question: {question}
SQLQuery:
"""

prompt = ChatPromptTemplate.from_template(template)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
db = SQLDatabase.from_uri("sqlite:///../database/northwind-SQLite3/dist/northwind.db")

def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

sql_response.invoke({"question": "How many employees are there?"})